<a href="https://colab.research.google.com/github/IsaacFigNewton/CSC-480/blob/main/Revised_Roomba_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Config

In [5]:
import networkx as nx
import json
import random
import numpy as np

contents = """7
5
_#a____
___#__#
*__#_*_
_______
____*__
"""

# save an offset map
offset_map = {
    # idx 0 => row offset
    # idx 1 => col offset
    "N": (-1, 0),
    "S": (1, 0),
    "E": (0, 1),
    "W": (0, -1)
}

## Helpers

## GameTree Class

In [16]:
class World:
  def __init__(self, file_contents:str):
    world_lst = file_contents.split("\n")
    self.num_cols = int(world_lst[0])
    self.num_rows = int(world_lst[1])
    self.grid = [[c for c in row] for row in world_lst[2:-1]]
    self.dirt_count = np.sum([np.sum([1 for c in row if c == "*"]) for row in self.grid])

    print("Successfully loaded world:")
    for row in self.grid:
      print(row)


  # ensure the move is not OOB and not blocked by an obstacle
  def is_legal_move(self, current_pos, proposed_pos):
    is_adjacent = (proposed_pos[0] - current_pos[0], proposed_pos[1] - current_pos[1]) in offset_map.values()
    is_OOB_row = proposed_pos[0] < 0 or proposed_pos[0] >= self.num_rows
    is_OOB_col = proposed_pos[1] < 0 or proposed_pos[1] >= self.num_cols
    is_blocked = self.grid[proposed_pos[0]][proposed_pos[1]] == "#"
    return not is_OOB_row and not is_OOB_col and not is_blocked

In [17]:
class Agent:
  def __init__(self, world: World):
    # vars for tracking position within game tree for DFS and UCS
    self.world = world
    self.stack = list()
    self.path = list()
    self.pos = self.get_bot_pos_from_grid()
    # configure the list of preferred moves
    self.children = dict()
    self.expand_children()


  def get_bot_pos_from_grid(self):
    for row in range(self.world.num_rows):
      for col in range(self.world.num_cols):
        if self.world.grid[row][col] == "a":
          return (row, col)
    raise ValueError("No bot found in the grid")


  def get_proposed_move_pos(self, move):
    diff = offset_map[move]
    return (
        self.pos[0] + diff[0],
        self.pos[1] + diff[1]
    )


  def expand_children(self):
    # create a list of possible moves
    possible_moves = {
        move: self.get_proposed_move_pos(move)
        for move in offset_map.keys()
    }
    # filter it down to a set of legal moves
    legal_moves = {
        move: new_pos for move, new_pos in possible_moves.items()
        if self.world.is_legal_move(self.pos, new_pos)
    }
    # filter it down to a set of preferred moves
    #   ie moves to locations/positions not previously visited
    for move, new_pos in legal_moves.items():
      if new_pos not in self.path:
        # add the move to the current node's dict of children
        self.children[move] = new_pos
        # add the child to the stack
        self.stack.append(new_pos)
    print(self.children)


  def select_action(self):
    # if the bot's over dirt, vacuum it up
    if self.world.grid[self.pos[0]][self.pos[1]] == "*":
      print("V")
      # update the game environment
      self.world.grid[self.pos[0]][self.pos[1]] = "_"
      dirt_count = dirt_count - 1
      print(f"dirt left: {dirt_count}")

    self.path.append(self.pos)
    # select the first available preferred move
    #   or pop an adjacent node from the stack
    #   or backtrack by 1 node
    chosen_move = list(self.children.items())[0]
    print(chosen_move)
    chosen_move = chosen_move or (
                      self.stack.pop()
                      if world.is_legal_move(self.pos, self.stack[-1])
                      else None
                  )
    print(chosen_move)
    chosen_move = chosen_move or self.path[-2]
    print(chosen_move)
    # update bot's location
    self.pos = chosen_move[1]

# Test DFS

In [18]:
world = World(contents)
agent = Agent(world)

Successfully loaded world:
['_', '#', 'a', '_', '_', '_', '_']
['_', '_', '_', '#', '_', '_', '#']
['*', '_', '_', '#', '_', '*', '_']
['_', '_', '_', '_', '_', '_', '_']
['_', '_', '_', '_', '*', '_', '_']
{'S': (1, 2), 'E': (0, 3)}


In [22]:
i = 10
while world.dirt_count > 0 and i > 0:
  agent.select_action()
  i -= 1

('S', (1, 2))
('S', (1, 2))
('S', (1, 2))
('S', (1, 2))
('S', (1, 2))
('S', (1, 2))
('S', (1, 2))
('S', (1, 2))
('S', (1, 2))
('S', (1, 2))
('S', (1, 2))
('S', (1, 2))
('S', (1, 2))
('S', (1, 2))
('S', (1, 2))
('S', (1, 2))
('S', (1, 2))
('S', (1, 2))
('S', (1, 2))
('S', (1, 2))
('S', (1, 2))
('S', (1, 2))
('S', (1, 2))
('S', (1, 2))
('S', (1, 2))
('S', (1, 2))
('S', (1, 2))
('S', (1, 2))
('S', (1, 2))
('S', (1, 2))
